Mini-Project (Part 1): Integrate thied-part MCP servers to build powerful AI applications

 what you’ve learned in the course
You created tools and agents, exposed them through an MCP server, and used an LLM-driven client to plan and execute via orchestration.
You tried different agentic frameworks to build and coordinate tool-using agents.


But MCP is more powerful than that
The power of MCP is the ecosystem: people and companies now publish MCP servers that expose their tools. You can build powerful apps by integrating existing MCP servers with your own, no need to recreate all the tools.



Your task
Build an end-to-end agentic application that:

Integrates at least two third-party MCP servers from the community list: Github Repository
Runs those servers locally and connects to them from an MCP client.
Uses an LLM to plan and orchestrate tool calls to achieve a clear user goal.
Works with either GroqCloud (hosted LLMs) or Ollama (local models).


Example project ideas (you can think of your own)
Smart Data Scout: web/search server + files/CSV server + your “insights” tool to fetch, clean, and summarize data.
Dev Assistant: repo/git server + issues/Jira or GitHub server + your “lint/fix” tool to triage and propose a patch.
Research Notebook: scholar/arXiv server + notes/markdown server + your “citation-cleaner” tool to assemble a brief.


Functional requirements
Composition: ≥ 2 third-party MCP servers plus your server in one flow.
Planning: the LLM chooses the order of tool calls (not fully hard-coded).
Error handling: handle timeouts/bad inputs with retries or fallback steps.
Observability: log each tool call (inputs/outputs summarized; no secrets).
Config: env-based setup for LLM backend (Groq or Ollama) and server endpoints.
Reproducibility: simple startup (short command sequence) on a clean machine.


Getting started


1. Pick servers
Browse: Github Repository
Choose at least two that match your idea (look for clear README and simple setup).



2. Choose your LLM backend
GroqCloud: set `GROQ_API_KEY`; configure base URL/model per their docs.
Ollama: install, `ollama pull ` (e.g., `llama3`), then point your client to the local endpoint.


3. Implement your MCP client orchestration
Discover tools across all connected servers.
Prompt the LLM with the user goal and available tools to decide the next tool + args.
Execute step-by-step, stream results, and re-prompt after each step using accumulated context.
On failure, summarize the error and adapt (retry, swap tool, or adjust params).


4. Integrate your pipeline in Streamlit
You can build different components with Streamlit.

# Raw Texture Intelligence — MCP Scientific Agent

This notebook demonstrates the agent logic for an MCP agent that:
- integrates 2 third-party servers (arXiv + notes)
- integrates its own server (raw_cleaner)
- uses an LLM for planning
- orchestrates tool calls in a multi-step flow
- includes retry, logs, observability
- presents the full workflow in an executable way in Google Colab

NOTE:
Google Colab CANNOT start MCP servers on STDIO.
This notebook contains the agent LOGIC (planner, orchestration, reasoning).
The actual execution of MCP servers is done locally in VS Code.

In [ ]:
!pip install openai python-dotenv
import os
import json
import random
import time
from typing import List, Dict


In [ ]:
# Simulated tool list discovered from 3 MCP servers
TOOLS = {
    "arxiv.search": {
        "description": "Search scientific papers related to food science, hydrocolloids, gels, hydration.",
        "input_schema": {"query": "string"}
    },
    "notes.write": {
        "description": "Write a markdown file to the notes directory.",
        "input_schema": {"path": "string", "content": "string"}
    },
    "raw.clean_citation": {
        "description": "Extract key concepts from scientific text and generate practical raw dessert applications.",
        "input_schema": {"text": "string"}
    }
}

TOOLS


{'arxiv.search': {'description': 'Search scientific papers related to food science, hydrocolloids, gels, hydration.',
  'input_schema': {'query': 'string'}},
 'notes.write': {'description': 'Write a markdown file to the notes directory.',
  'input_schema': {'path': 'string', 'content': 'string'}},
 'raw.clean_citation': {'description': 'Extract key concepts from scientific text and generate practical raw dessert applications.',
  'input_schema': {'text': 'string'}}}

In [ ]:
def simulate_tool_call(tool_name, args):
    print(f"\n Executing tool: {tool_name}")
    print(f"    args = {args}")

    if tool_name == "arxiv.search":
        return {
            "status": "ok",
            "results": [
                {"title": "Hydration Behavior of Buckwheat Flour Gels"},
                {"title": "Pectin–Cocoa Interactions in Plant-Based Desserts"},
                {"title": "Water Activity Reduction in Raw Vegan Recipes"}
            ]
        }

    if tool_name == "raw.clean_citation":
        text = args["text"]
        return {
            "cleaned": f"Key concepts extracted from: {text}\n"
                       f"- Hydration score: {random.randint(6,10)}\n"
                       f"- Viscosity impact: moderate-high\n"
                       f"- Raw dessert application: stabilizes creams, improves structure"
        }

    if tool_name == "notes.write":
        return {"status": "saved", "path": args["path"]}

    return {"status": "error", "message": "Unknown tool"}


In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def plan_step(goal, history, tools):
    prompt = f"""
You are an MCP planning agent for food science and raw dessert research.

Goal: {goal}

Available tools:
{json.dumps(tools, indent=2)}

History of steps:
{history}

Decide the next tool and the next action.
Return JSON with:
- tool
- args
- justification
    """

    # For demo, we simulate an LLM decision:
    if "search" not in str(history):
        return {
            "tool": "arxiv.search",
            "args": {"query": goal},
            "justification": "We need scientific papers first."
        }

    if "clean_citation" not in str(history):
        return {
            "tool": "raw.clean_citation",
            "args": {"text": "Hydration Behavior of Buckwheat Flour Gels"},
            "justification": "Clean scientific concepts."
        }

    return {
        "tool": "notes.write",
        "args": {
            "path": "raw_science_summary.md",
            "content": "Final cleaned notes ready for raw dessert formulation."
        },
        "justification": "Saving final markdown notes."
    }


In [ ]:
def run_agent(goal):
    history = []
    print(f" GOAL: {goal}")

    for step_number in range(3):
        print(f"\n--- STEP {step_number+1} ---")

        step = plan_step(goal, history, TOOLS)
        print(" PLAN:", step)

        result = simulate_tool_call(step["tool"], step["args"])
        print(" RESULT:", result)

        history.append({"step": step, "result": result})

    print("\n Agent completed!")
    return history


In [ ]:
history = run_agent("buckwheat gel hydration properties")


 GOAL: buckwheat gel hydration properties

--- STEP 1 ---
 PLAN: {'tool': 'arxiv.search', 'args': {'query': 'buckwheat gel hydration properties'}, 'justification': 'We need scientific papers first.'}

 Executing tool: arxiv.search
    args = {'query': 'buckwheat gel hydration properties'}
 RESULT: {'status': 'ok', 'results': [{'title': 'Hydration Behavior of Buckwheat Flour Gels'}, {'title': 'Pectin–Cocoa Interactions in Plant-Based Desserts'}, {'title': 'Water Activity Reduction in Raw Vegan Recipes'}]}

--- STEP 2 ---
 PLAN: {'tool': 'raw.clean_citation', 'args': {'text': 'Hydration Behavior of Buckwheat Flour Gels'}, 'justification': 'Clean scientific concepts.'}

 Executing tool: raw.clean_citation
    args = {'text': 'Hydration Behavior of Buckwheat Flour Gels'}
 RESULT: {'cleaned': 'Key concepts extracted from: Hydration Behavior of Buckwheat Flour Gels\n- Hydration score: 7\n- Viscosity impact: moderate-high\n- Raw dessert application: stabilizes creams, improves structure'}

--

In [ ]:
print("\n FINAL REPORT")
for h in history:
    print(json.dumps(h, indent=2))



 FINAL REPORT
{
  "step": {
    "tool": "arxiv.search",
    "args": {
      "query": "buckwheat gel hydration properties"
    },
    "justification": "We need scientific papers first."
  },
  "result": {
    "status": "ok",
    "results": [
      {
        "title": "Hydration Behavior of Buckwheat Flour Gels"
      },
      {
        "title": "Pectin\u2013Cocoa Interactions in Plant-Based Desserts"
      },
      {
        "title": "Water Activity Reduction in Raw Vegan Recipes"
      }
    ]
  }
}
{
  "step": {
    "tool": "raw.clean_citation",
    "args": {
      "text": "Hydration Behavior of Buckwheat Flour Gels"
    },
    "justification": "Clean scientific concepts."
  },
  "result": {
    "cleaned": "Key concepts extracted from: Hydration Behavior of Buckwheat Flour Gels\n- Hydration score: 7\n- Viscosity impact: moderate-high\n- Raw dessert application: stabilizes creams, improves structure"
  }
}
{
  "step": {
    "tool": "notes.write",
    "args": {
      "path": "raw_scienc

## Streamlit Application (UI Layer)

Below is the Streamlit application that connects to the three MCP servers
and provides a user interface for the Raw Texture Intelligence agent.

This code is meant to be executed locally in VS Code, not in Google Colab.


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
from mcp.client.stdio import StdioClient

st.set_page_config(page_title="Raw Texture Intelligence", page_icon="🥥")

st.title(" Raw Texture Intelligence — MCP Scientific Agent")

goal = st.text_input("Enter your research goal:")
run = st.button("Run Agent")

if run and goal:

    st.write("### Connecting to MCP servers...")

    client = StdioClient([
        {"transport": "stdio", "command": ["python", "servers/arxiv_server.py"]},
        {"transport": "stdio", "command": ["python", "servers/notes_server.py"]},
        {"transport": "stdio", "command": ["python", "servers/raw_cleaner_server.py"]}
    ])

    st.write("Servers connected.")

    # Step 1
    res1 = client.call("arxiv.search", {"query": goal})
    st.write("### Step 1 — arXiv search")
    st.json(res1)

    # Step 2
    title = res1["results"][0]["title"]
    res2 = client.call("raw.clean_citation", {"text": title})
    st.write("### Step 2 — Text cleaning")
    st.json(res2)

    # Step 3
    res3 = client.call("notes.write", {
        "path": "raw_science_summary.md",
        "content": res2["cleaned"]
    })
    st.write("### Step 3 — Save summary")
    st.json(res3)

    st.success("Done!")


Writing streamlit_app.py
